In [1]:
import os

In [2]:
%pwd

'd:\\Develop\\Pipline\\Face_recognitions\\notebook'

In [3]:
os.chdir("../")


In [4]:
%pwd

'd:\\Develop\\Pipline\\Face_recognitions'

In [5]:
from deepface import DeepFace
from pathlib import Path

In [17]:
img_path=Path('artifacts/data_ingestion/dataset/train/5/5.jpg')
img_path2=Path('artifacts/data_ingestion/dataset/train/5/412.jpg')

In [9]:
embedding_objs = DeepFace.represent(img_path = img_path ,model_name="Facenet512",detector_backend="mtcnn")

1/1 [==============================] - 0s 295ms/step


In [10]:
embedding_objs

[{'embedding': [-1.7831395864486694,
   0.6163805723190308,
   -0.04494704306125641,
   -0.1325152963399887,
   0.9571791291236877,
   -0.10707896947860718,
   -0.06879990547895432,
   0.3514883816242218,
   1.0519720315933228,
   0.2003207504749298,
   0.953106164932251,
   -0.8066203594207764,
   0.543879508972168,
   -1.5485670566558838,
   -0.1774260550737381,
   -0.36015579104423523,
   0.8056014180183411,
   1.02352774143219,
   -0.3421138823032379,
   -1.0020434856414795,
   -0.6530301570892334,
   -0.16872099041938782,
   1.68328058719635,
   0.941188633441925,
   0.11234540492296219,
   -1.0708534717559814,
   1.6994792222976685,
   -0.385707825422287,
   0.03923971205949783,
   0.13919484615325928,
   -0.4319112002849579,
   -0.318881094455719,
   -0.8375611305236816,
   0.276917964220047,
   0.3635674715042114,
   1.391933560371399,
   1.75677490234375,
   -0.5738216638565063,
   -0.7246654033660889,
   -0.17084725201129913,
   0.7622718811035156,
   0.30164873600006104,
   

In [15]:
from  deepface.modules.recognition import __find_bulk_embeddings 

In [18]:
path=[img_path,img_path2]

In [19]:
# pkl=__find_bulk_embeddings(path,
#     "Facenet512",
#     (224, 224),
#     "mtcnn",
#     True,
#      True,
#      0,
#      "base",
#      False
# )

Finding representations:   0%|          | 0/2 [00:00<?, ?it/s]

1/1 [==============================] - 0s 31ms/step


Finding representations:  50%|█████     | 1/2 [00:02<00:02,  2.65s/it]

1/1 [==============================] - 0s 26ms/step


Finding representations: 100%|██████████| 2/2 [00:05<00:00,  2.63s/it]


entity

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class Train_Representation_Model_Config:
    model_root_dir: Path
    represent_file:Path
    train_faces_dir:Path
    model_name: str
    face_detector_backend: str
    target_size: tuple
    align: bool
    enforce_detection: bool
    expand_percentage: int
    normalization:str
    silent: bool

    

src configaration management

In [32]:
from face_recognitions.constants import *
from face_recognitions.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_YAML_PATH,
                 params_filepath=PARAMS_YAML_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_train_representation_model_config(self)->Train_Representation_Model_Config:
        config=self.config.train_model
        params=self.params
        print(config)
        create_directories([config.model_root_dir])

        train_representation_model_config=Train_Representation_Model_Config(
            model_root_dir=config.model_root_dir,
            represent_file= config.represent_file,
            train_faces_dir=config.train_faces_dir,
            model_name= params.MODEL_NAME,
            face_detector_backend=params.FACE_DETECTOR_BACKEND,
            target_size=params.TARGET_SIZE, 
            align=params.ALIGN,
            enforce_detection=params.ENFORCE_DETECTION,
            expand_percentage=params.EXPAND_PERCENTAGE,
            normalization=params.NORMALIZATION,
            silent=params.SILENT
        )

        return train_representation_model_config



Training model Component

In [ ]:
import os
import pickle
import time
from pathlib import Path
from typing import List

import pandas as pd
from tqdm import tqdm

from deepface.modules import representation, detection, modeling
from deepface.models.FacialRecognition import FacialRecognition

from face_recognitions import logger

class Train_Representation_Model :
    def __init__(self,config:Train_Representation_Model_Config) :
        self.config=config

    def list_of_image_path(self,path:Path):

        """
        List images in a given path
        Args:
            path (str): path's location
        Returns:
            images (list): list of exact image paths
        """
        images = []
        for r, _, f in os.walk(path):
            for file in f:
                if file.lower().endswith((".jpg", ".jpeg", ".png")):
                    exact_path = os.path.join(r, file)
                    images.append(exact_path)
        return images[0:10]


    def find_bulk_embeddings(
        self,
        employees: List[Path],
        model_name: str = "VGG-Face",
        target_size: tuple = (224, 224),
        detector_backend: str = "opencv",
        enforce_detection: bool = True,
        align: bool = True,
        expand_percentage: int = 0,
        normalization: str = "base",
        silent: bool = False,
    ):
        """
        Find embeddings of a list of images

        Args:
            employees (list): list of exact image paths

            model_name (str): facial recognition model name

            target_size (tuple): expected input shape of facial recognition model

            detector_backend (str): face detector model name

            enforce_detection (bool): set this to False if you
                want to proceed when you cannot detect any face

            align (bool): enable or disable alignment of image
                before feeding to facial recognition model

            expand_percentage (int): expand detected facial area with a
                percentage (default is 0).

            normalization (bool): normalization technique

            silent (bool): enable or disable informative logging
        Returns:
            representations (list): pivot list of embeddings with
                image name and detected face area's coordinates
        """
        representations = []
        for employee in tqdm(
            employees,
            desc="Finding representations",
            disable=silent,
        ):
            try:
                img_objs = detection.extract_faces(
                    img_path=employee,
                    target_size=target_size,
                    detector_backend=detector_backend,
                    grayscale=False,
                    enforce_detection=enforce_detection,
                    align=align,
                    expand_percentage=expand_percentage,
                )
            except ValueError as err:
                logger.error(
                    f"Exception while extracting faces from {employee}: {str(err)}"
                )
                img_objs = []

            if len(img_objs) == 0:
                logger.warn(f"No face detected in {employee}. It will be skipped in detection.")
                representations.append((employee, None, 0, 0, 0, 0))
            else:
                for img_obj in img_objs:
                    img_content = img_obj["face"]
                    img_region = img_obj["facial_area"]
                    embedding_obj = representation.represent(
                        img_path=img_content,
                        model_name=model_name,
                        enforce_detection=enforce_detection,
                        detector_backend="skip",
                        align=align,
                        normalization=normalization,
                    )

                    img_representation = embedding_obj[0]["embedding"]
                    representations.append((
                        employee,
                        img_representation,
                        img_region["x"],
                        img_region["y"],
                        img_region["w"],
                        img_region["h"]
                        ))

        return representations


    def representations(self):
       
        tic = time.time() 

        # -------------------------------
        if os.path.isdir(self.config.train_faces_dir) is not True:
            raise ValueError("Passed train_faces_dir does not exist!")

        model: FacialRecognition = modeling.build_model(self.config.model_name)
        target_size = model.input_shape
        # ---------------------------------------

        representations = []

        df_cols = [
        "identity",
        f"{self.config.model_name}_representation",
        "target_x",
        "target_y",
        "target_w",
        "target_h",
        ]

        # Ensure the proper pickle file exists
        if not os.path.exists(self.config.represent_file):
            with open(self.config.represent_file, "wb") as f:
                pickle.dump([], f)

        # Load the representations from the pickle file
        with open(self.config.represent_file, "rb") as f:
            representations = pickle.load(f)

        # Check if the representations are out-of-date
        if len(representations) > 0:
            if len(representations[0]) != len(df_cols):
                raise ValueError(
                    f"Seems existing {self.config.represent_file} is out-of-the-date."
                    "Please delete it and re-run."
                )
            pickled_images = [representation[0] for representation in representations]
        else:
            pickled_images = []

        # Get the list of images on storage
        storage_images =self.list_of_image_path(path=self.config.train_faces_dir)

        # Enforce data consistency amongst on disk images and pickle file
        must_save_pickle = False
        new_images = list(set(storage_images) - set(pickled_images)) # images added to storage
        old_images = list(set(pickled_images) - set(storage_images)) # images removed from storage

        if not self.config.silent and (len(new_images) > 0 or len(old_images) > 0):
            logger.info(f"Found {len(new_images)} new images and {len(old_images)} removed images")

        # remove old images first
        if len(old_images)>0:
            representations = [rep for rep in representations if rep[0] not in old_images]
            must_save_pickle = True

        # find representations for new images
        if len(new_images)>0:
            representations += self.find_bulk_embeddings(
                employees=new_images,
                model_name=self.config.model_name,
                target_size=target_size,
                detector_backend=self.config.face_detector_backend,
                enforce_detection=self.config.enforce_detection,
                align=self.config.align,
                normalization=self.config.normalization,
                silent=self.config.silent,
            ) # add new images
            must_save_pickle = True

        if must_save_pickle:
            with open(self.config.represent_file, "wb") as f:
                pickle.dump(representations, f)
            if not self.config.silent:
                logger.info(f"There are now {len(representations)} representations in representation.pkl")

        # Should we have no representations bailout
        if len(representations) == 0:
            if not self.config.silent:
                toc = time.time()
                logger.info(f"find function duration {toc - tic} seconds")
            return []


In [66]:

try:
    config=ConfigurationManager()
    train_representation_model_config=config.get_train_representation_model_config()
    train_representation_model=Train_Representation_Model(config=train_representation_model_config)
    train_representation_model.representations()
except Exception as e:
    raise e


[2024-02-27 16:29:14,107: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-27 16:29:14,112: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-27 16:29:14,115: INFO: common: created directory at: artifacts]
{'model_root_dir': 'artifacts/model', 'represent_file': 'artifacts/model/representation.pkl', 'train_faces_dir': 'artifacts/data_ingestion/dataset/train'}
[2024-02-27 16:29:14,118: INFO: common: created directory at: artifacts/model]
[2024-02-27 16:29:14,261: INFO: 1750340772: Found 0 new images and 20 removed images]
[2024-02-27 16:29:14,269: INFO: 1750340772: There are now 10 representations in representation.pkl]


In [67]:
try:
    obj2=ConfigurationManager()
    aa=obj2.get_train_representation_model_config()
    modelpath=aa.represent_file
    representations = []
    df_cols = [
        "identity",
        f"{aa.model_name}_representation",
        "target_x",
        "target_y",
        "target_w",
        "target_h",
    ]
    # Load the representations from the pickle file
    with open(modelpath, "rb") as f:
        representations = pickle.load(f)
    
    # Check if the representations are out-of-date
    if len(representations) > 0:
        if len(representations[0]) != len(df_cols):
            raise ValueError(
                f"Seems existing {modelpath} is out-of-the-date."
                "Please delete it and re-run."
            )
        pickled_images = [representation[0] for representation in representations]
    else:
        pickled_images = []

    # ----------------------------
    # now, we got representations for facial database
    df = pd.DataFrame(
        representations,
        columns=df_cols,
    )

except Exception as e :
    raise e

[2024-02-27 16:29:18,623: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-27 16:29:18,625: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-27 16:29:18,625: INFO: common: created directory at: artifacts]
{'model_root_dir': 'artifacts/model', 'represent_file': 'artifacts/model/representation.pkl', 'train_faces_dir': 'artifacts/data_ingestion/dataset/train'}
[2024-02-27 16:29:18,625: INFO: common: created directory at: artifacts/model]


In [68]:
print(df)

                                            identity  \
0  artifacts/data_ingestion/dataset/train\102\311...   
1  artifacts/data_ingestion/dataset/train\102\145...   
2  artifacts/data_ingestion/dataset/train\102\123...   
3  artifacts/data_ingestion/dataset/train\102\372...   
4  artifacts/data_ingestion/dataset/train\102\162...   
5  artifacts/data_ingestion/dataset/train\102\381...   
6  artifacts/data_ingestion/dataset/train\102\289...   
7  artifacts/data_ingestion/dataset/train\102\240...   
8  artifacts/data_ingestion/dataset/train\102\112...   
9  artifacts/data_ingestion/dataset/train\102\126...   

                           Facenet512_representation  target_x  target_y  \
0  [-1.2840473651885986, 0.10071158409118652, 0.9...       270       185   
1  [-0.25313863158226013, 1.1500993967056274, 0.6...       314       194   
2  [-0.28061434626579285, 2.3476295471191406, 0.9...       189       140   
3  [-1.879409670829773, 0.7437683939933777, 1.133...       241       169   
4  